In [1]:
import pandas as pd

In [2]:
trip = pd.read_csv(r'R:\SoundCastDocuments\2014Estimation\Files_From_Mark_2014\gps_weights_11_3_16\formatted\skims_attached\tripP14_w.dat')

In [3]:
# Reclassify trip purpose for 4k purposes
# Use Daysim User's Guide for data dictionary for trip records

# Home-Based Work (HBW) Trips, directly from home-to-work and work-to-home
trip.ix[(trip['opurp'] == 0) & (trip['dpurp'] == 1), '4k_purp'] = 'HBW'
trip.ix[(trip['opurp'] == 1) & (trip['dpurp'] == 0), '4k_purp'] = 'HBW'

# Home-Based shopping
trip.ix[(trip['opurp'] == 0) & (trip['dpurp'] == 5), '4k_purp'] = 'Home-Based Shopping'
trip.ix[(trip['opurp'] == 5) & (trip['dpurp'] == 0), '4k_purp'] = 'Home-Based Shopping'

# Home-Based Other
trip.ix[(trip['opurp'] == 0) & (trip['dpurp'].isin([3,4,6,7,8,9,10])), '4k_purp'] = 'Home-Based Other'
trip.ix[(trip['opurp'].isin([3,4,6,7,8,9,10]) & (trip['dpurp'] == 0)), '4k_purp'] = 'Home-Based Other'

# NHB Work-to-Other
trip.ix[(trip['opurp'] == 1) & (trip['dpurp'] != 0) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'
trip.ix[(trip['opurp'] != 0) & (trip['dpurp'] == 1) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'

# # NHB Other-to-Other (Destination and Origins are neither work nor home)
trip.ix[(trip['opurp'] != 1) & (trip['dpurp'] != 0) & (trip['opurp'] != 0) 
        & (trip['dpurp'] != 1) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB OtO'

### Weighted Average Travel Time by Trip Purpose

In [11]:
trip['wt_travtime'] = trip['travtime']*trip['trexpfac']
trip.groupby('4k_purp').sum()['wt_travtime']/trip.groupby('4k_purp').sum()['trexpfac']

4k_purp
HBW                    25.261620
Home-Based Other       15.098397
Home-Based Shopping    12.644990
NHB OtO                13.014079
NHB WtO                16.035133
dtype: float64

In [13]:
# Split HBW trips by income groups

In [14]:
hh = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-households.xlsx',
                  sheetname='Data')

# Household income
# Note that exact ranges from 2014 do not match 2006
# old ranges: 30, 60, 90+
# new ranges: 35, 75, 100+
hh['income_crossclass'] = hh['hh_income_detailed_imp']
hh.ix[hh['hh_income_detailed_imp'] <= 3, 'income_crossclass'] = '1'    # $35k  
hh.ix[(hh['hh_income_detailed_imp'] > 3) & (hh['hh_income_detailed_imp'] <= 5),    # $35-75k
           'income_crossclass'] = '2' 
hh.ix[(hh['hh_income_detailed_imp'] > 5) & (hh['hh_income_detailed_imp'] <= 6),    # $75-100k
           'income_crossclass'] = '3' 
hh.ix[hh['hh_income_detailed_imp'] >= 7, 'income_crossclass'] = '4'    # >$100k

In [18]:
df = trip[trip['4k_purp'] == 'HBW']

df = pd.merge(df, hh, how='left', left_on='hhno', right_on='hhid')
df.groupby(['income_crossclass']).sum()['wt_travtime']/df.groupby(['income_crossclass']).sum()['trexpfac']

income_crossclass
1    21.550708
2    24.181637
3    27.982392
4    26.273059
dtype: float64